# **Control SDP using the Tango devices**

Controlling Science Data Processor (SDP) using Tango devices allows to directly interact with the devices such as querying,
changing attributes and executing commands directly to Tango.

The Tango devices in the SDP are:
   - The SDP controller Tango device is designed to provide the overall control of the SDP. The commands it receives cause
     the other SDP services to be stopped or started, and its attributes report on the overall state of the system.
   - The SDP subarray Tango device is the principal means by which processing is initiated in the SDP.

If you get stuck you can look at the official documentation pages:
 - [SDP Integration](https://developer.skao.int/projects/ska-sdp-integration/en/latest/index.html)
 - [SDP Local Monitoring and Control](https://developer.skao.int/projects/ska-sdp-lmc/en/latest/index.html)
 - [SDP Processing Scripts](https://developer.skao.int/projects/ska-sdp-script/en/latest/)
 - [SDP on the Developer Portal](https://developer.skao.int/en/latest/projects/area/sdp.html)

If you still don't find the answer to your questions, contact us on Slack: #......

In [ ]:
import os
from tango import DeviceProxy, EventType

# specify here the namespace to connect in this cluster
KUBE_NAMESPACE = "<update-with-ns!!!>"

# set the name of the databaseds service
DATABASEDS_NAME = "databaseds-tango-base"

# finally set the TANGO_HOST
os.environ["TANGO_HOST"] = f"{DATABASEDS_NAME}.{KUBE_NAMESPACE}.svc.cluster.local:10000"

In [ ]:
lsdev

In [ ]:
d = DeviceProxy('test-sdp/subarray/01')

In [ ]:
d.state()

In [ ]:
d.obsState()

In [ ]:
d.obsState()

In [ ]:
config_sbi = '''
{
   "interface":"https://schema.skao.int/ska-sdp-assignres/0.3",
   "eb_id":"eb-test-20210630-00000",
   "max_length":21600.0,
   "scan_types":[
      {
         "scan_type_id":"science",
         "reference_frame":"ICRS",
         "ra":"02:42:40.771",
         "dec":"-00:00:47.84",
         "channels":[
            {
               "count":13824,
               "start":0,
               "stride":2,
               "freq_min":0.35e9,
               "freq_max":0.368e9,
               "link_map":[
                  [
                     0,
                     0
                  ],
                  [
                     200,
                     1
                  ],
                  [
                     744,
                     2
                  ],
                  [
                     944,
                     3
                  ]
               ]
            }
         ]
      },
      {
         "scan_type_id":"calibration",
         "reference_frame":"ICRS",
         "ra":"12:29:06.699",
         "dec":"02:03:08.598",
         "channels":[
            {
               "count":13824,
               "start":0,
               "stride":2,
               "freq_min":0.35e9,
               "freq_max":0.368e9,
               "link_map":[
                  [
                     0,
                     0
                  ],
                  [
                     200,
                     1
                  ],
                  [
                     744,
                     2
                  ],
                  [
                     944,
                     3
                  ]
               ]
            }
         ]
      }
   ],
   "mccs": {
      "station_ids": [0, 1, 2, 3, 4, 5]
   },
   "processing_blocks":[
      {
         "pb_id":"pb-test-20211111-00000",
         "workflow":{
            "kind":"realtime",
            "name":"vis-receive",
            "version":"0.5.1"
         },
         "parameters": {
            "plasmaEnabled": true,
            "transmission": {
               "channels_per_stream": 6912,
               "rate": "10416667"
            },
            "reception": {
               "layout": "http://127.0.0.1:80/model/default/ska1_low/layout",
               "schedblock": "/mnt/data/sb-test.json",
               "num_channels": 13824,
               "channels_per_stream": 6912,
               "continuous_mode": true
            },
            "pvc": {
               "name": "receive-data"
            },
            "plasma_parameters": {
               "initContainers":[
                  {
                     "name":"existing-output-remover",
                     "image":"artefact.skao.int/ska-sdp-realtime-receive-modules:2.1.0",
                     "command":["rm", "-rf", "/mnt/data/output.ms"],
                     "volumeMounts":[
                        {
                           "mountPath":"/mnt/data",
                           "name":"receive-data"
                        }
                     ]
                  }
               ],
               "extraContainers":[
                  {
                     "name":"plasma-processor",
                     "image":"artefact.skao.int/ska-sdp-realtime-receive-modules:2.1.0",
                     "command":[
                        "plasma-mswriter",
                        "-s",
                        "/plasma/socket",
                        "--max_payloads",
                        "12",
                        "--use_plasma_ms",
                        "False",
                        "/mnt/data/output.ms"
                     ],
                     "volumeMounts":[
                        {
                           "name":"plasma-storage-volume",
                           "mountPath":"/plasma"
                        },
                        {
                           "mountPath":"/mnt/data",
                           "name":"receive-data"
                        }
                     ]
                  },
                  {
                     "name": "tmlite-server",
                     "image": "artefact.skao.int/ska-sdp-tmlite-server:0.3.0"
                  }
               ]
            }
         }
      }
   ]
}
'''

In [ ]:
d.AssignResources(config_sbi)

In [ ]:
d.obsState

In [ ]:
d.receiveAddresses

In [ ]:
d.Configure('{"scan_type": "science"}')

In [ ]:
d.obsState

In [ ]:
d.Scan('{"id": 1}')

In [ ]:
d.obsState

In [ ]:
d.EndScan()

In [ ]:
d.obsState

In [ ]:
d.End()

In [ ]:
d.obsState

In [ ]:
d.ReleaseResources()

In [ ]:
d.obsState

In [ ]:
d.Off()

In [ ]:
d.state()

More details about each of the SDP Subarray commands can be found here [here](https://developer.skao.int/projects/ska-sdp-lmc/en/latest/sdp_subarray.html)